<a href="https://colab.research.google.com/github/omkarpat/EmpatheticDialoguesEmotionDetection/blob/master/Complement_Emotion_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 245\ Project

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 245 Project


In [3]:
!ls Data

'Data Exploration.ipynb'
 data_fixed_train_complement_analysis.csv
 data_fixed_train.json
 data_sample_100.json
 data_sample_10_complement_analysis.csv
 data_sample_10.json
 data_sample_10_processed.json
 data_sample_fixed_processed_final.json
 data_sample_fixed_processed_model1_final.csv
 data_sample_fixed_processed_model2_final.csv
 data_sample_fixed_processed_model3_final.csv
 data_sample_fixed_processed_model4_final.csv
 fixed
 fixed_test.csv
 fixed_test.json
 fixed_train_516.csv
 fixed_valid.csv
 fixed_valid.json
'informative words.ipynb'
 model1_predictions_test.csv
 model1_predictions_train.csv
 model1_predictions_valid.csv
 model2_predictions_test.csv
 model2_predictions_train.csv
 model2_predictions_valid.csv
 model3_predictions_test.csv
 model3_predictions_train.csv
 model3_predictions_valid.csv
 model4_predictions_test.csv
 model4_predictions_train.csv
 model4_predictions_valid.csv
'Model Metrics.gsheet'
 Raw
 test_fixed_processed_final.json
 test_fixed_processed_model1_fina

In [4]:
!pip install transformers

     |████████████████████████████████| 675kB 2.7MB/s 
     |████████████████████████████████| 3.8MB 16.1MB/s 
     |████████████████████████████████| 890kB 28.2MB/s 
     |████████████████████████████████| 1.2MB 38.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b9300982873f8908680c8a32a880d682568dc6d8d97ab65a040814d01bc71562
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
dataset_name = "data_fixed_train_complement_analysis"
speaker_model_name = "model3"
listener_model_name = "model2"
model_type = "emotion" #pos_neg or emotion

In [6]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
import json
with open("Data/data_sample_10.json") as infile:
  data = json.load(infile)
print(data['1022'])

{'prompt': 'We (family friends) planned a holiday trip to go for the last weekend. I was preparing for that like packing dress and getting ready to it. Unfortunately one of my cousin was unable to come due to his personal reasons. So we just cancelled the trip. I was feeling sad for this trip cancellation. ', 'emotion': 'sad', 'dialog': [{'speaker': 43, 'utterance': 'My holiday trip has cancelled. I was feeling low for it. '}, {'speaker': 167, 'utterance': 'That is bad news and hard to deal with.'}, {'speaker': 43, 'utterance': 'yeah. Waiting for the next holiday.'}, {'speaker': 167, 'utterance': 'I sure hope you have a wonderful next Holiday trip very soon.'}, {'speaker': 43, 'utterance': 'thank you! have a nice day'}]}


In [0]:
print(data.keys())
print(data['2103'])

dict_keys(['1022', '2103', '4072', '9710', '11290', '11572', '13042', '13485', '18577', '21184'])
{'prompt': 'My cousin bought a really pretty shirt that I had been wanting.', 'emotion': 'jealous', 'dialog': [{'speaker': 226, 'utterance': 'My cousin bought a really pretty shirt that I had been wanting'}, {'speaker': 10, 'utterance': 'My cousin bought a really pretty shirt that I had been wanting'}, {'speaker': 226, 'utterance': 'that is not fun_comma_ does that mean you cannot get it now?'}, {'speaker': 10, 'utterance': "No I can but I didn't want her to have it and not me."}, {'speaker': 226, 'utterance': 'that is no fun_comma_ does that mean you cannot buy it now?'}, {'speaker': 10, 'utterance': "No I can but I didn't want her to have it and not me."}]}


In [0]:
with open("Data/data_fixed_train.json","r") as infile:
  data = json.load(infile)
  with open("Data/data_complement_analysis.csv","w") as outfile:
    outfile.write("emotion|speaker_utterance|listener_utterance\n")
    for key in data.keys():
      dialog = data[key]['dialog']
      emotion = data[key]['emotion']
      i = 0
      while i < len(dialog):
        speaker_utterance = str(dialog[i]['utterance']).replace("|","").replace("_comma_",",")
        listener_utterance = str(dialog[i+1]['utterance']).replace("|","").replace("_comma_",",") if i + 1 < len(dialog) else ""
        line = "%s|%s|%s\n" % (emotion, speaker_utterance, listener_utterance)
        outfile.write(line)
        #print(line)
        i += 2

In [7]:
import pandas as pd
import numpy as np
train_filename = "Data/" + dataset_name + ".csv"
train_df = pd.read_csv(train_filename, sep='|', error_bad_lines=False)
train_df = train_df.dropna()
train_df

,emotion,speaker_utterance,listener_utterance
0,sentimental,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju..."
1,sentimental,This was a best friend. I miss her.,Where has she gone?
2,sentimental,We no longer talk.,Oh was this something that happened because of...
3,sad,"During christmas a few years ago, I did not ge...","Wow, that must be terrible, I cannot imagine, ..."
4,sad,Since that day christmas has not been a good t...,"Wow, I am sorry to hear that, I wish I could m..."
...,...,...,...
43904,nostalgic,"Yeah! It's weird, malls feel like ghost towns,...",I live in a very large city on the east coast....
43906,anticipating,ive been waiting for months for the new madden...,Heck yeah! I myself have never been into madde...
43907,anticipating,"i love 2k, proam is the best!","I've been playing it since I was a kid, but I ..."
43908,faithful,I'm faithful I'm going down the right career p...,"that is a great feeling, what are you pusuing?"


In [0]:
category2index = {
    "invalid":0,
    "sentimental":1,
    "sad":2,
    "hopeful":3,
    "joyful":4,
    "impressed":5,
    "terrified":6,
    "proud":7,
    "afraid":8,
    "grateful":9,
    "lonely":10,
    "surprised":11,
    "anxious":12,
    "guilty":13,
    "trusting":14,
    "nostalgic":15,
    "excited":16,
    "disgusted":17,
    "caring":18,
    "furious":19,
    "angry":20,
    "prepared":21,
    "disappointed":22,
    "faithful":23,
    "confident":24,
    "apprehensive":25,
    "jealous":26,
    "embarrassed":27,
    "devastated":28,
    "annoyed":29,
    "ashamed":30,
    "content":31,
    "anticipating":32
}
index2category = {}
for key, value in category2index.items():
  index2category[value] = key

def category2label(category):
  for key in category2index.keys():
    if key == category:
      return category2index[key]
  return 0

def label2emotion(labels):
  emotion_list = []
  for label in labels:
    emotion_list.append(index2category[label])
  return emotion_list

In [9]:
print('Number of training sentences: {:,}\n'.format(train_df.shape[0]))
if model_type == "emotion":
  train_df['orignial_label'] = train_df['emotion'].apply(lambda x: int(category2label(x.lower())))

# Display 10 random rows from the data.
train_df.sample(10)

Number of training sentences: 40,245



,emotion,speaker_utterance,listener_utterance,orignial_label
41188,grateful,I am so lucky to have the things I have. My 10...,"Wow, it sounds like you have a lot to be grate...",9
40201,lonely,"I guess, I got through it",Maybe next weekend will be better for you!,10
12586,grateful,my friend helped me move last week,"Cool, how long did that take?",9
39168,furious,Last week I cleaned my house when my kids were...,That is so satisfying. I hope they don't mess ...,19
6628,ashamed,I cheated on my math test today.,Oh man. Must be feeling pretty guilty by now.,30
13348,terrified,The best I can do is incorporate humor into my...,"I'm sure you'll be fine, just try to practice ...",6
35470,surprised,"I was an editor, so I designed and wrote up th...",That is wonderful. I use to love English class...,11
35543,disappointed,"That's a good idea, I didn't think about that ...",Im always living on a budget so even if you co...,22
13606,anxious,"Yeah I feel the same way, then it ends up bein...","Similar to most things, yea. What was wrong wi...",12
6540,disappointed,They're leaving too much work behind for me to...,See. They will only hinder the overall perform...,22


In [0]:
speaker_sentences_train = train_df.speaker_utterance.values
listener_sentences_train = train_df.listener_utterance.values
labels_train = train_df.orignial_label.values

In [11]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

longer_list = []
for index, sent in enumerate(speaker_sentences_train):
  if len(tokenizer.encode(sent, add_special_tokens=True)) > 500:
    longer_list.append(index)
speaker_sentences_train = np.delete(speaker_sentences_train, longer_list)

longer_list = []
for index, sent in enumerate(listener_sentences_train):
  if len(tokenizer.encode(sent, add_special_tokens=True)) > 500:
    longer_list.append(index)
listener_sentences_train = np.delete(listener_sentences_train, longer_list)
from transformers import BertTokenizer

def tokenize(sentences, labels):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.

      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 256,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.from_numpy(labels)

  return(input_ids, attention_masks, labels)

speaker_input_ids_train, speaker_attention_masks_train, speaker_labels_train = tokenize(speaker_sentences_train, labels_train)
listener_input_ids_train, listener_attention_masks_train, listener_labels_train = tokenize(listener_sentences_train, labels_train)

Loading BERT tokenizer...


In [12]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
speaker_dataset_train = TensorDataset(speaker_input_ids_train, speaker_attention_masks_train, speaker_labels_train)
listener_dataset_train = TensorDataset(listener_input_ids_train, listener_attention_masks_train, listener_labels_train)

print('{:>5,} training samples'.format(len(speaker_dataset_train)))

40,245 training samples


In [13]:
from transformers import BertForSequenceClassification, BertTokenizer
output_dir = "./" + model_type + "_" + speaker_model_name + "_save/"

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

print(model_type + "_" + speaker_model_name + " loaded...")

emotion_model3 loaded...


In [14]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

def predict(dataset):

  # Create the DataLoader for  dataset.
  prediction_data = dataset
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


  # Prediction on test set

  print('Predicting labels for {:,} sentences...'.format(len(dataset)))

  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels, speaker_utterance = [], [], []

  # Predict 
  for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  print('DONE.')
  return predictions, true_labels

speaker_train_predictions, speaker_train_true_labels = predict(speaker_dataset_train)

Predicting labels for 40,245 sentences...


KeyboardInterrupt: ignored

In [1]:
def predictions2labels(predictions):
  for i in range(len(predictions)):
    if i == 0:
      pred_flat = np.argmax(predictions[i], axis=1).flatten()
    else:
      pred_flat = np.append(pred_flat, np.argmax(predictions[i], axis=1).flatten())
  return label2emotion(pred_flat)
train_df['emotion_speaker'] = pd.DataFrame(predictions2labels(speaker_train_predictions), columns=['emotion_speaker'])
train_df.sample(10)

NameError: ignored

In [1]:
del model
torch.cuda.empty_cache()

NameError: ignored

In [29]:
from transformers import BertForSequenceClassification, BertTokenizer
output_dir = "./" + model_type + "_" + listener_model_name + "_save/"

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

print(model_type + "_" + listener_model_name + " loaded...")

emotion_model2 loaded...


In [30]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

def predict(dataset):

  # Create the DataLoader for  dataset.
  prediction_data = dataset
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


  # Prediction on test set

  print('Predicting labels for {:,} sentences...'.format(len(dataset)))

  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []

  # Predict 
  for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  print('DONE.')
  return predictions, true_labels

listener_train_predictions, listener_train_true_labels = predict(listener_dataset_train)

Predicting labels for 40,245 sentences...
DONE.


In [31]:
train_df['emotion_listener'] = pd.DataFrame(predictions2labels(listener_train_predictions), columns=['emotion_listener'])
train_df.sample(10)

,emotion,speaker_utterance,listener_utterance,orignial_label,emotion_speaker,emotion_listener
24462,devastated,"It is really sad, I had to call and order a ne...",Scary situation loosing a card...Hopefully you...,28,proud,surprised
28117,sentimental,They live in the city. I just like the memorie...,Nice I'm in TX and my fam's in MN. I try to v...,1,excited,prepared
21167,content,"Absolutely! I can get so much done, but I real...",That's not surprising! Are you thinking about ...,31,embarrassed,grateful
527,content,I recently had a child and my marriage is thri...,Great news. Kids are always an exciting adven...,31,joyful,caring
34962,apprehensive,I'm starting a new job in September!,That's awesome. What kind of job?,25,grateful,trusting
10686,prepared,I am ready to go on Jeopardy.,when do you go?,21,afraid,angry
28520,nostalgic,Yeah. Im pretty sure they think I was born in ...,"Right, it's hard to imagine there was a time b...",15,surprised,anxious
28963,trusting,I let my cousin look after my dog whilst I wen...,"Oh nice, how did it go?",14,apprehensive,prepared
40588,devastated,I am so hurt right now.,Forget him. There are soo many girls out there.,28,NaN,NaN
2112,apprehensive,I have been preparing for grad school and it i...,"I just got in a few months ago, best of luck m...",25,devastated,sad
